In [42]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from scipy import stats

In [2]:
!pip install gdown

In [3]:
import gdown
url = 'https://drive.google.com/uc?id=1UOJcE-YEfPqhF65Gc1wrZx9N16kLLmf6'
output = 'train_test_zip'
gdown.download(url, output)

In [4]:
!unzip train_test_zip

In [7]:
os.chdir('/kaggle/working')
os.mkdir('inc_rn')
os.chdir('inc_rn')
url = 'https://drive.google.com/uc?id=1WwKL9qFj8Qxj-bselgg-5x5NyRCZsoh2'
output = 'inc_rn'
gdown.download(url, output)
!unzip inc_rn
os.chdir('..')
print('{} loaded.'.format(output))

In [8]:
os.mkdir('rn')
os.chdir('rn')
url = 'https://drive.google.com/uc?id=1e4xa2sJFGQMfyoKKuuS-YKKhRL2KDO8z'
output = 'rn'
gdown.download(url, output)
!unzip rn
os.chdir('..')
print('{} loaded.'.format(output))

In [9]:
os.mkdir('sm')
os.chdir('sm')
url = 'https://drive.google.com/uc?id=13Q4eYoPFVlVdaJdHhhHJKMJAbDPV4Ez1'
output = 'sm'
gdown.download(url, output)
!unzip sm
os.chdir('..')
print('{} loaded.'.format(output))

In [10]:
os.mkdir('sm2')
os.chdir('sm2')
url = 'https://drive.google.com/uc?id=1C1dqu7B1dEE5vrUcSB6NPg0U-sPSlgQK'
output = 'sm2'
gdown.download(url, output)
!unzip sm2
os.chdir('..')
print('{} loaded.'.format(output))

In [53]:
import os
os.getcwd()

In [11]:
os.chdir('/kaggle/working')
inc_rn = tfk.models.load_model('inc_rn')
rn     = tfk.models.load_model('rn')
sm     = tfk.models.load_model('sm')
sm2    = tfk.models.load_model('sm2')

In [18]:
url = 'https://drive.google.com/uc?id=1eRQTLO3bRq4vMC8stjkkphJsPj3u3CIC'
output = 'ensemble_model.h5'
gdown.download(url, output)

In [19]:
!ls

In [26]:
class CustomLayer(tfk.layers.Layer):
    
    def __init__(self, units = 1, **kwargs):
        self.units = 1
        
        super(CustomLayer, self).__init__(**kwargs)
        
        
    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({
            'units': self.units,
        })
        return config
    
    def call(self, inputs):
        from collections import Counter
        os.chdir('/kaggle/working')
        models = [inc_rn, rn, sm, sm2]
        models_preds = []
        for model in models:
            model_output = tf.math.argmax(model(inputs), axis=1)
            models_preds.append(model_output)

        return models_preds

    
model = tfk.models.load_model('ensemble_model.h5', custom_objects = {'CustomLayer': CustomLayer})

In [30]:
# Creating an instance of ImageDataGenerator without Data Augmentation
noaug_train_data_gen = ImageDataGenerator(preprocessing_function=tfk.applications.resnet50.preprocess_input)
valid_data_gen       = ImageDataGenerator(preprocessing_function=tfk.applications.resnet50.preprocess_input)
test_data_gen        = ImageDataGenerator(preprocessing_function=tfk.applications.resnet50.preprocess_input)

os.chdir('/')

dataset_dir  = r'kaggle/working'
training_dir = os.path.join(dataset_dir,'train')
val_dir      = os.path.join(dataset_dir,'val')
seed = 42

noaug_train_gen = noaug_train_data_gen.flow_from_directory(directory=training_dir,
                                                           target_size=(256,256),
                                                           color_mode='rgb',
                                                           classes=None,
                                                           class_mode='categorical',
                                                           batch_size=1,
                                                           shuffle=True,
                                                           seed=seed)

valid_gen = valid_data_gen.flow_from_directory(directory=val_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None,
                                               class_mode='categorical',
                                               batch_size=1,
                                               shuffle=False, # no need to shuffle validation set
                                               seed=seed)

In [32]:
os.chdir('/kaggle/working/val')

In [34]:
os.chdir('/')
preds = model.predict(valid_gen)

In [36]:
preds

In [45]:
models_preds = np.array(preds)
models_preds = np.transpose(models_preds, (1, 0))
models_preds = stats.mode(models_preds, axis=1)[0]
models_preds = np.squeeze(models_preds)
len(models_preds)

In [49]:
acc   = 0
pos   = 0
total = models_preds.shape[0]
for i in range(len(models_preds)):
    if models_preds[i] == valid_gen.labels[i]:
        pos += 1

acc = pos/total
print("accuracy is: {}".format(acc))